In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import math

Group Assignment
Team Number: 16
Team Member Names: Umer.C, Prithvi.S, David.Z
Team Strategy Chosen: Market Beat
Contribution Declaration
The following team members made a meaningful contribution to this assignment:

Umer.C, Prithvi.S, David.Z

We will generate a portfolio with 15 stocks. We purposely made the number to be on the lower end between 10 and 25 since our objective is to beat the market and we thus do not care as much for diversification

In [ ]:
# CFM 101 - Group Assignment 2025
# Robo-Advising Challenge
# Competition Goal: Market Beat - Highest return above the benchmark average

# ============================================================================
# CONSTANTS
# ============================================================================

BUY_PORTFOLIO_DATE = datetime.strptime("2025-11-21", "%Y-%m-%d").date()


print("Dear TA, if there is a division by 0 error, there wasn't a small-cap stock that went through the stock filter in the csv")

SMALL_CAP = 2 # this can be made bigger to fix the issue above
BIG_CAP = 10

INITIAL_CAPITAL = 1000000  # CAD
VOLUME_CHECK_START = '2024-10-01'
VOLUME_CHECK_END = '2025-09-30'
MIN_STOCKS = 10
MAX_STOCKS = 25
MAX_WEIGHT = 0.15
MAX_SECTOR_WEIGHT = 0.40
MIN_TRADING_DAYS = 18

TRAINING_DAY_BEGIN = "2023-11-21"
TRAINING_DAY_END = "2024-11-22"

YEAR_AFTER_TRAINING_DAY_BEGIN = "2024-11-21"
YEAR_AFTER_TRAINING_DAY_END = "2025-11-22"

PORTFOLIO_DATA_BEGIN = "2024-11-21"
PORTFOLIO_DATA_END = "2025-11-22"
RISK_FREE_RATE = 2/12
N_STOCKS = 15


# convenient dictionary to get the cad-usd exchange rate given a date 
cadusd = yf.Ticker("CADUSD=X")
cadusd_df = cadusd.history(start="2023-01-01", end = "2025-11-22")
cadusd_df.index = cadusd_df.index.date

CADUSD = dict()

idx = 0

for date in cadusd_df.index:
    CADUSD[date]=cadusd_df['Close'].iloc[idx]
    idx+=1

Dear TA, if there is a division by 0 error, there wasn't a small-cap stock that went through the stock filter in the csv
Make the SMALL_CAP variable bigger to fix this


In [41]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

# example of a US and Canadian Stock so that we can getting the trading days
# of the Canadian and US Market
us_ticker_ex = yf.Ticker("AAPL")
cad_ticker_ex = yf.Ticker("Shop.to")

def first_trading_days_of_each_month (start_date, end_date):
    us_ticker_ex_hist = us_ticker_ex.history(start= start_date, end = end_date)
    us_ticker_ex_hist.index = us_ticker_ex_hist.index.date
    cad_ticker_ex_hist = cad_ticker_ex.history(start= start_date, end = end_date)
    cad_ticker_ex_hist.index = cad_ticker_ex_hist.index.date

   
    valid_dates = np.intersect1d(us_ticker_ex_hist.index, cad_ticker_ex_hist.index)
    
    first_trading_days= []
    prev_month = 0
    for date in valid_dates:
        if date.month != prev_month:
            first_trading_days.append(date)
            prev_month = date.month

    return first_trading_days


# this function creates a dataframe of a portfolio's returns, given
# an array of tickers, the dates to keep track for the value of the portfolio,
# and the weight of each stock in the portfolio

def create_df(stocks, dates, distribution):
    start_date = dates[0]
    end_date = dates[len(dates)-1]+pd.Timedelta(days=1)
    
    # creating a dataframe for each ticker with its close price at every first trading day of the month
    stocks_df = []

    
    for ticker in stocks:
        temp_df = ticker.history(start=start_date, end=end_date)  
        temp_df.index = temp_df.index.date
        count = 0

        df_dict = {'Date': dates, 
               'Close': []}

        for i in range (len (temp_df)):
            if temp_df.index[i] == dates[count]:
                close = temp_df['Close'].iloc[i]
                df_dict['Close'].append(close)
                count += 1


        stock_df = pd.DataFrame(df_dict)
        stock_df = stock_df.set_index("Date")
      
        stocks_df.append(stock_df)

    # finding out how many shares in each stock will be bought
    shares_in_stocks = []

    money_in_each_stock = []
    for i in range (len(stocks)):
        money_in_each_stock.append(distribution[i]*INITIAL_CAPITAL)
    
    for i in range (len(stocks)):
        close = stocks_df[i]['Close'].iloc[0]
        if ticker.info['currency'] == 'USD':
            close /= CADUSD[dates[0]]

        shares_in_stocks.append(money_in_each_stock[i]/close)

    # creating a dataframe that stores the portfolio value at every first trading day of the month
    stock_portfolio = {
        "Date": dates,
        "Portfolio Value": []}
    
    for i in range (len(dates)):
        portfolio_value = 0
        for j in range(len(stocks)):
            close = stocks_df[j]['Close'].iloc[i]
            if ticker.info['currency'] == 'USD':
                close /= CADUSD[dates[i]]
            portfolio_value += close*shares_in_stocks[j]
            
        stock_portfolio["Portfolio Value"].append(portfolio_value)
    
    stock_portfolio = pd.DataFrame(stock_portfolio)
    stock_portfolio = stock_portfolio.set_index("Date")
    stock_portfolio['Percentage Returns']= stock_portfolio['Portfolio Value'].pct_change()*100
    stock_portfolio.drop(index=stock_portfolio.index[0], inplace = True)
    return stock_portfolio


def calculate_technical_features(df):
    features = {}
    
    # Returns
    features['return_200d'] = df['Close'].pct_change(200).iloc[-1]
    
    # Volatility
    features['volatility_200'] = -1*df['Close'].pct_change().rolling(200).std().iloc[-1]

    
    # Moving averages
    features['sma_200'] = df['Close'].rolling(200).mean().iloc[-1]
    features['price_to_sma200'] = df['Close'].iloc[-1] / features['sma_200'] 
    
    # Momentum
    features['rsi'] = calculate_rsi(df['Close'], 14)
    features['momentum'] = df['Close'].iloc[-1] / df['Close'].iloc[-200] - 1 
        
    return features

def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.iloc[-1]


def check_volume_requirement(ticker, start, end):
    data = ticker.history(start=start, end=end)


    # gets all the months with >= 18 trading days 
    valid_months = set()

    cur_mo = data.index[0].month

    days = 0
    for date in data.index:
        if date.month == cur_mo:
            days +=1 
        if date.month != cur_mo or date == data.index[-1]:
            if days>=MIN_TRADING_DAYS:
                valid_months.add(cur_mo)
            cur_mo = date.month
            days = 0

    # finding the mean of volumes on trading days in a valid month

    idx = 0
    num_days = 0
    sum = 0

    for date in data.index:
        if date.month in valid_months:
            sum += data['Volume'].iloc[idx]
            num_days += 1

    return float(sum)/num_days >= 5000



def get_market_cap(ticker):
    market_cap = ticker.info['marketCap']
    
    currency = ticker.info['currency']
    if currency == 'USD':
        market_cap *= CADUSD[BUY_PORTFOLIO_DATE]
    
    return market_cap / 1e9 

def norm (col):
    new_col = (col-col.mean())/col.std()
    return new_col


After scoring our stocks, we will standardize the scores into z-scores. Unfortunately, z-scores can be negative and we wish to convert the z_scores into positive values. A simple approach is to use e^x. However, if x is decently large, e^x will become absurdly large. To fix this, we will use the softplus function, where softplus(x) = ln(1+e^x).

We will sort our stocks by highest to lowest scores. We will go through the list of stocks from top to bottom so that the stocks with the highest scores will be put into our portfolio. 

We now have our list of 15 stocks. We also want stocks with the higher softplus-transformed z-scores to have higher weights in our portfolio. To do this, we will first let every selected stock have the minimum weight. From here, we will have some excess weights leftover. 

We will first distribute the excess weights to the sectors.

To illustrate how we do this, it's best to use an example:
    Let's say we have an array of 3 numbers: [0.3, 0.2, 0.1]
    We need to make the sum of these numbers 1, so we have an excess weight of 0.4

    After distributing the excess weights, we wish for the ending numbers to maintain roughly
    the same relative proportions as when they started

    Another constraint we have is that no ending number can be > 0.4

    We also have a corresponding array describing how much of the excess weight each
    number should get:  [0.5, 0.3, 0.2]

    If the previous constraint didn't exist, we would just do:

    [0.3, 0.2, 0.1] -> [0.3+0.5*0.4, 0.2+0.3*0.4, 0.1+0.2*0.4] -> roughly [0.5, 0.32, 0.18]

    Obviously 0.5 > 0.4, so there is a problem. 

    We will use the following algorithm to deal with this:

    First iteration:

        The best we can do is convert 0.3 to 0.4

        [0.3, 0.2, 0.1] -> [0.4, 0.2+0.3*0.4, 0.1+0.2*0.4] -> [0.4, 0.32, 0.18]

    Second iteration

        There is still an excess weight of 0.1 remaining, we will split it among the first two elements.

        So our array representing how much of the excess weight each number should get will become just:

        [0.3, 0.2]

        we will divide each element by the sum which is 0.3+0.2 = 0.5

        Hence the array becomes:

        [0.6, 0.4]

        from here:

        [0.4, 0.32, 0.18] -> [0.4, 0.32*0.6*0.1, 0.18 + 0.4*0.1] = [0.4, 0.38, 0.22]


We will implement this algorithm to distribute the excess weight to sectors. And then for each sector, we will distribute the excess weight available for the sector to the stocks within that sector.

The array representing how much of the excess weight each sector receives is calculated by dividing the sum of all tickers' scores in that sector by the total score of all tickers across all sectors.

The array representing how much of a sector’s allocated excess weight each ticker receives is calculated by dividing the ticker’s score by the sum of all tickers' scores in that sector

In [42]:
def softplus (n):
    return math.log(1+math.e**n)

def weights (ticker_score):

    # ticker_score is already sorted in descending order by scores
    # each element of ticker_score is an array of 2 element: [ticker, score_of_ticker]

    scores = []
    n = min(len(ticker_score), 15)

    min_weight_per_stock = 1.0/(2*n)
    excess_weight = 1 -n*min_weight_per_stock
    sum_softplus = 0


    # z_score
    for i in range (len(ticker_score)) :
        scores.append(ticker_score[i][1])

    mean = np.average(scores)
    std = np.std(scores)

    ticker_z_score = dict()

    for i in range (len(ticker_score)):
        ticker = ticker_score[i][0]
        z_score = (ticker_score[i][1]-mean)/std
        ticker_z_score[ticker] = z_score


    selected_tickers = []
    num_stocks_selected = 0
    sectors = dict()

    has_small_cap = False
    has_big_cap = False


    for pair in ticker_score:
        ticker = pair[0]
        ticker_sector = ticker.info['sector']
        left_to_choose = n - num_stocks_selected-1 # how much stocks we have left to choose if we pick this stock
        market_cap = get_market_cap(ticker)
        small_cap = market_cap < SMALL_CAP
        big_cap = market_cap > BIG_CAP

        if not ticker_sector in sectors:
            sectors[ticker_sector] = []
        elif (len(sectors[ticker_sector])+1)*min_weight_per_stock > MAX_SECTOR_WEIGHT: #check if too many stocks in the sector
            continue

        max_weight = 0
    
        # makes sure the sum of the maximum weight of each sector won't be less than 1
        for sector in sectors:
            length = len(sectors[sector])
            if (sector == ticker_sector):
                length+=1

            if (length == 0):
                continue

            max_weight_sector = min(MAX_SECTOR_WEIGHT, MAX_WEIGHT*length)
            max_weight += max_weight_sector

        max_weight+= (left_to_choose)*MAX_WEIGHT

        if max_weight < 1:
            continue

        # makes sures the portfolio has one large-cap and one small-cap stock

        if not has_small_cap and not has_big_cap and left_to_choose == 1 and not small_cap and not big_cap:
            continue
        if not has_small_cap and left_to_choose == 0 and not small_cap:
            continue
        if not has_big_cap and left_to_choose == 0 and not big_cap:
            continue

        if big_cap:
            has_big_cap=True
        if small_cap:
            has_small_cap=True

        sectors[ticker_sector].append(ticker)
        
        num_stocks_selected += 1
        selected_tickers.append(ticker)

        if num_stocks_selected == n:
            break

    # softplus_score 
    ticker_softplus_score = dict()

    for ticker in selected_tickers:
        score = softplus(ticker_z_score[ticker])
        ticker_softplus_score[ticker] = score
        sum_softplus += score


    weights = dict()

    for ticker in ticker_softplus_score.keys():
        weights[ticker] = min_weight_per_stock


    add_sector_weights = dict()
    sector_sum_scores = dict()

    #distribute excess weights across sectors
    #there's a lot of stuff with 1e-10 because of decimal imprecision
    
    for sector in sectors.keys():
        add_sector_weights[sector] = 0
        sector_sum_scores[sector] = 0
        for ticker in sectors[sector]:
            sector_sum_scores[sector] += ticker_softplus_score[ticker]

    while excess_weight > 1e-10:
        sum = 0
        for sector in sectors.keys():
            current_weight = len(sectors[sector])*min_weight_per_stock+add_sector_weights[sector]
            if abs(current_weight-min (MAX_SECTOR_WEIGHT, len(sectors[sector])*MAX_WEIGHT)) < 1e-10: # if already max out
                continue
            sum += sector_sum_scores[sector]

        for sector in sectors.keys():
            current_weight = len(sectors[sector])*min_weight_per_stock+add_sector_weights[sector]
            if (sum==0):
                print(len(sectors[sector]))
                print(sectors)
            toAdd = min( min(MAX_SECTOR_WEIGHT, len(sectors[sector])*MAX_WEIGHT) -current_weight, sector_sum_scores[sector]/sum*excess_weight)
            add_sector_weights[sector] += toAdd
            excess_weight -= toAdd

    #distribute excess weights across tickers 

    for sector in sectors.keys():
        while add_sector_weights[sector] > 1e-10:
            sum = 0
            for ticker in sectors[sector]:
                if abs(weights[ticker]-MAX_WEIGHT) < 1e-10: # if already maxed out
                    continue
                sum += ticker_softplus_score[ticker]

            for ticker in sectors[sector]:
                score = ticker_softplus_score[ticker]
                toAdd = min(MAX_WEIGHT-weights[ticker], score/sum*add_sector_weights[sector])
                weights[ticker] += toAdd
                add_sector_weights[sector] -= toAdd

    return weights

In [ ]:
# ============================================================================
# Filtering
# ============================================================================

# Load tickers
tickers_df = pd.read_csv('Tickers.csv', header = None) # example csv didn't have a header 
tickers_list = tickers_df.iloc[:, 0].tolist()
print(f"Total tickers loaded: {len(tickers_list)}")


for i in range (len(tickers_list)):
    tickers_list[i]=yf.Ticker(tickers_list[i])

# Filter stocks based on volume requirement
print("\nFiltering stocks by volume requirement...")
valid_tickers = []
for ticker in tickers_list:
    try:
        if check_volume_requirement(ticker, VOLUME_CHECK_START, VOLUME_CHECK_END):
            valid_tickers.append(ticker)
        else:
            print(f"{ticker} did not pass volume filter")
    except:
        pass

print(f"Stocks passing volume filter: {len(valid_tickers)}")

$AGN: possibly delisted; no timezone found
$CELG: possibly delisted; no timezone found
$MON: possibly delisted; no timezone found
$RTN: possibly delisted; no timezone found


Total tickers loaded: 42

Filtering stocks by volume requirement...
Stocks passing volume filter: 37


The following assesses each remaining stock using four technical variables that are validated by research in finance. 

The first is price momentum, which assesses whether or not a stock has shown long-run price power. This is just the current price, subtracted by the price 200 trading days ago. This has long been demonstrated to persistently outperform and outsmart investors via persistent capital inflows into winning assets. 

The second is volatility, which is negatively weighted to favor companies that provide stable long-run stock market returns and risk-adjusted risk-returns. This volatility is only measured via the Standard Error Deviation daily treatment to ensure that excessive or abnormal price fluctuations are discouraged. 

The SMA-200 is calculated by averaging the past 200 daily closing prices, and it acts as a widely respected long-term trend filter in institutional trading. Firms that trade above this moving average are in healthy upward trends, whereas those priced below are generally experiencing long-term declines and are therefore excluded to avoid “value traps” that could continue falling.

The fourth variable is the Relative Strength Index (RSI), a momentum oscillator used to evaluate whether an asset has become overbought. Its calculation involves computing daily price differences over a 14-day period and separating them into average gains and losses. These are used to form a ratio (RS), which is then converted into an index between 0 and 100. While values above 70 indicate an unsustainably strong buying surge, values between 30 and 70 signify a healthy, stable upward trend. By penalizing overly high RSI readings, the program prevents selecting assets at the peak of speculative buying phases where reversals are likely.

To assess how important each variable is, the program conducts thousands of simulation trials on  randomly generated weights. For each simulation trial, it generates a hypothetical portfolio and analyzes its performance based on its Sharpe Ratio, which documents the return on investment for every unit of volatility. This enables it to select the weighting pattern most suitable to the current market environment. The weights with the highest Sharpe ratio is then used to score and rank every stock. 

In [ ]:


# Prepare data for finding optimal weights for scoring each individual stock
col_names = ['ticker', 'momentum', 'price_to_sma200', 'volatility_200', 'rsi']
stock_features_dict = {'ticker': [],
                       'momentum': [], 
                       'price_to_sma200': [], 
                       'volatility_200': [],
                       'rsi': []}

for ticker in valid_tickers:
    data = ticker.history(start= TRAINING_DAY_BEGIN, end = TRAINING_DAY_END)
    
    if data is None or len(data) < 200:
        print(f"No Data for {ticker}")
        continue
    
    try:
        features = calculate_technical_features(data)
        features['ticker'] = ticker

        for name in col_names:
            stock_features_dict[name].append(features[name])

    except Exception as e:
        print(f"  Error processing {ticker}: {e}")
        continue



# standardize each variable to z-score
stock_features_df = pd.DataFrame(stock_features_dict)
stock_features_df.set_index('ticker', inplace=True)
stock_features_df['momentum_norm'] = norm(stock_features_df['momentum'])
stock_features_df['price_to_sma200_norm'] = norm(stock_features_df['price_to_sma200'])
stock_features_df['volatility_200_norm'] = norm(stock_features_df['volatility_200'])
stock_features_df['rsi_norm']= norm(stock_features_df['rsi'])

TRIALS = 1000
variables = ['momentum_norm', 'price_to_sma200_norm', 'volatility_200_norm', 'rsi_norm']
n = len(variables)

optimal_weights = []
max_sharpe_ratio = -100

for i in range (TRIALS):
    var_weights = np.random.rand(n) # randomly generate weights for each trial
    var_weights = var_weights/var_weights.sum()

    ticker_score = []

    for i in range (len(stock_features_df)):
        score = 0
        for j in range(n):
            score += var_weights[j]*stock_features_df[variables[j]].iloc[i]
        ticker_score.append([stock_features_df.index[i], score])


    # sort by descending order of score
    ticker_score = sorted(ticker_score, key=lambda x: x[1], reverse=True)


    # make the portfolio, find the sharpe ratio 
    stock_weights = weights(ticker_score)

    tickers = []
    distribution = []

    
    dates = first_trading_days_of_each_month(YEAR_AFTER_TRAINING_DAY_BEGIN, YEAR_AFTER_TRAINING_DAY_END)

    for ticker in stock_weights.keys():
        tickers.append(ticker)
        distribution.append(stock_weights[ticker])


    df = create_df(tickers, dates, distribution)

    sharpe = (df['Percentage Returns'].mean()-RISK_FREE_RATE)/df['Percentage Returns'].std()

    if sharpe > max_sharpe_ratio:
        max_sharpe_ratio=sharpe
        optimal_weights= var_weights


# ============================================================================
# Building Portfolio 
# ============================================================================

stock_features_dict = {'ticker': [],
                       'momentum': [], 
                       'price_to_sma200': [], 
                       'volatility_200': [],
                       'rsi': []}



for ticker in valid_tickers:
    data = ticker.history(start= PORTFOLIO_DATA_BEGIN, end = PORTFOLIO_DATA_END)
    

    features = calculate_technical_features(data)
    features['ticker'] = ticker

    for name in col_names:
        stock_features_dict[name].append(features[name])

   


stock_features_df = pd.DataFrame(stock_features_dict)
stock_features_df.set_index('ticker', inplace=True)
stock_features_df['momentum_norm'] = norm(stock_features_df['momentum'])
stock_features_df['price_to_sma200_norm'] = norm(stock_features_df['price_to_sma200'])
stock_features_df['volatility_200_norm'] = norm(stock_features_df['volatility_200'])
stock_features_df['rsi_norm']= norm(stock_features_df['rsi'])


ticker_score = []

for i in range (len(stock_features_df)):
    score = 0
    for j in range(n):
        score += optimal_weights[j]*stock_features_df[variables[j]].iloc[i]
    ticker_score.append([stock_features_df.index[i], score])



ticker_score = (sorted(ticker_score, key=lambda x: x[1], reverse= True))

distribution = weights(ticker_score)


final_portfolio_df_dict = {'Ticker': [],
                     'Price': [],
                     'Currency': [],
                     'Shares': [],
                     'Weight': [],
                     'Value': []}


for ticker in distribution.keys():
    price = ticker.history(start= "2025-11-21", end = "2025-11-22")['Close'].iloc[-1]
    currency = ticker.info['currency']
    weight = distribution[ticker]
    value = weight*INITIAL_CAPITAL

    if currency == 'USD':
        price /= CADUSD[BUY_PORTFOLIO_DATE]
    



    final_portfolio_df_dict['Ticker'].append(ticker.info['symbol'])
    final_portfolio_df_dict['Price'].append(price)
    final_portfolio_df_dict['Currency'].append(currency)
    final_portfolio_df_dict['Shares'].append(value/price)
    final_portfolio_df_dict['Weight'].append(weight)
    final_portfolio_df_dict['Value'].append(value)

# ============================================================================
# FINAL OUTPUT
# ============================================================================
final_portfolio = pd.DataFrame(final_portfolio_df_dict)


small_cap = False
big_cap = True
for ticker in distribution.keys():
    if (get_market_cap(ticker) < 2):
        small_cap = True
    if (get_market_cap(ticker) > 10):
        big_cap = True


transaction_fee = min(2.15, final_portfolio['Shares'].sum()*0.001)/CADUSD[BUY_PORTFOLIO_DATE]



final_portfolio['Value'] -= transaction_fee/len(distribution)
final_portfolio['Shares'] = final_portfolio['Value']/final_portfolio['Price']
final_portfolio.index += 1

print(final_portfolio)



print(f"Existence of small-cap stock (< {SMALL_CAP} billion): {small_cap}")
print(f"Existence of large-cap stock (> {BIG_CAP} billion): {big_cap}")


print(f"Sum of weights: {final_portfolio['Weight'].sum()}")
print(f"Portfolio Value: ${final_portfolio['Value'].sum():.2f} CAD")
print(f"Transaction Fee:  ${transaction_fee:.2f} CAD")

print("if the weights don't add up to 1 or the portfolio value isn't around a million, there wasn't a small-cap stock that went through the stock filter in the csv")



     Ticker        Price Currency       Shares    Weight          Value
1       CAT   775.621882      USD   146.074229  0.113299  113298.368631
2       LLY  1493.244339      USD    83.513916  0.124707  124706.682330
3      BIIB   247.018731      USD   385.440433  0.095211   95211.006569
4     TD.TO   115.589996      CAD   765.008537  0.088428   88427.333959
5      ABBV   332.946859      USD   228.181024  0.075972   75972.155173
6       MRK   137.755569      USD   451.551201  0.062204   62203.692645
7     RY.TO   211.380005      CAD   309.206127  0.065360   65359.992692
8      AAPL   382.561961      USD   151.142734  0.057822   57821.460758
9        KO   102.795296      USD   522.991390  0.053761   53761.054690
10       BK   149.972636      USD   351.078607  0.052652   52652.184322
11        C   139.080134      USD   352.226222  0.048988   48987.670293
12  SHOP.TO   208.279999      CAD   207.651185  0.043250   43249.588569
13      AXP   497.264359      USD    88.236882  0.043877   43877

In [47]:
sectors_weights = dict()

for ticker in distribution.keys():
    sector = ticker.info['sector']
    if not sector in sectors_weights:
        sectors_weights[sector] = 0
    sectors_weights[sector] += distribution[ticker]

    


print('-'*20)
for sector in sectors_weights.keys():
    print(f"{sector} {sectors_weights[sector]}")

print('-'*20)

--------------------
Industrials 0.11329857060509743
Healthcare 0.3580943446120793
Financial Services 0.2993052474852227
Technology 0.13590648080389583
Consumer Defensive 0.09339535621571549
--------------------


In [48]:
final_portfolio[['Ticker', 'Shares']].to_csv('Stocks_Group_16.csv')